# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Caching Strategies

**Subtopic** - Implementing a distributed cache simulation using multiprocessing or threading

**Use Case** - Develop a Python script that simulates a distributed cache using Python's multiprocessing or threading module. The simulation should be able to store, retrieve, and delete data from the cache. Ensure the cache is thread-safe and processes can access it concurrently without data corruption.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- Develop a Python script that simulates a distributed cache using Python's threading module.
- The simulation should be able to store, retrieve, and delete data from the cache.
- Ensure the cache is thread-safe and processes can access it concurrently without data corruption.

Input Format:
- PUT <key> <value>
- GET <key>
- DELETE <key>
- EXIT

Input Constraints:
- <key> must be a non-empty alphanumeric string with a maximum length of 64 characters.
- <value> must be a printable ASCII string with a maximum length of 256 characters.
- Keys are case-sensitive.
- A maximum of 10000 operations may be issued.
- Only one operation per line is allowed.
- Each input must contain exactly the expected number of space-separated arguments. Any command with missing or extra arguments is invalid and must result in ERROR.

Output Format:
- For GET <key>: print the value if found, otherwise print NOT_FOUND
- For DELETE <key>: print DELETED if the key existed and was removed, otherwise print NOT_FOUND
- For PUT <key> <value>: print STORED
- For EXIT: print BYE and terminate the program (Allow running threads to finish, but prevent new threads after it).
- For any invalid/malformed command: print ERROR

Function Signature:
```python
def simulate_distributed_cache(commands: list[str]) -> list[str]:

Explanation:
commands: A list of input command strings like "PUT user1 Alice", "GET user1", etc.
Returns: A list of output strings corresponding to each command's result. Outputs must be collected in a thread-safe manner.
```

Example
```python
Input:
PUT user1 Alice
PUT user2 Bob
GET user1
DELETE user2
GET user2
EXIT

Output:
STORED
STORED
Alice
DELETED
NOT_FOUND
BYE
```

# Requirements
Explicit Requirements:
- Each command must be executed in a separate thread to simulate concurrent access.
- The cache must support PUT, GET, DELETE, and EXIT operations.
- The cache must be thread-safe for concurrent access.
- The cache should store string key-value pairs.
- Each operation must produce appropriate output (e.g., STORED, NOT_FOUND).
- The system must terminate cleanly on EXIT command.
- Outputs must be returned in the same order as the input commands. Each command should still execute in a separate thread, but the final result list must reflect the input order.

Implicit Requirements:
- Concurrent threads must not corrupt shared data (e.g., race conditions must be avoided).
- Multiple threads may issue operations simultaneously.
- The implementation must use Python's threading module.
- Memory usage should be efficient; unused keys should be removable.
- Inputs must be parsed and validated according to constraints.

Solution Expectations:
- Implement a thread-safe cache using Python's threading primitives (e.g., Lock, RLock).
- Use a shared data structure (e.g., dict) to store key-value pairs safely.
- Provide a function that processes a sequence of cache commands and returns corresponding outputs.
- Ensure that concurrent access to the cache does not result in data races or inconsistent states.
- Maintain correct functional behavior for PUT, GET, DELETE, and EXIT commands.
- Handle invalid operations gracefully via printing errors.
- Respect all input constraints and produce outputs exactly as specified.
- The system should simulate concurrency using threads internally, even if the input is sequential.
- All thread interactions (reads/writes/deletes) must be safely synchronized.
- Implementations should be capable of handling up to 10,000 commands efficiently. Validation for exceeding this limit is not required.

Edge Cases and Behavior:
- GET on a non-existent key  
  - Input: GET unknown  
  - Output: NOT_FOUND

- DELETE on a non-existent key  
  - Input: DELETE unknown  
  - Output: NOT_FOUND

- PUT with an existing key  
  - Input:  
    - PUT user1 Alice  
    - PUT user1 Alicia  
  - Output:  
    - STORED  
    - STORED (value should be updated to "Alicia")

- Empty string as key or value  
  - Input: PUT "" value or PUT key ""  
  - Output: Invalid (should be rejected per constraint)

- Key exceeds 64 characters  
  - Input: PUT <long_key> value  
  - Output: Invalid (should be rejected per constraint)

- Value exceeds 256 characters  
  - Input: PUT key <very_long_value>  
  - Output: Invalid (should be rejected per constraint)

- Case sensitivity  
  - Input:  
    - PUT Key1 Value  
    - GET key1  
  - Output:  
    - STORED  
    - NOT_FOUND (keys are case-sensitive)

- Concurrent PUT and GET on same key  
  - Behavior: Must not result in inconsistent or partial reads.

- Concurrent DELETE and GET on same key  
  - Behavior: Either the value is returned or NOT_FOUND - never corrupted.

- EXIT command in the middle of processing  
  - Input:  
    - PUT a A  
    - EXIT  
    - GET a  
  - Output:  
    - STORED  
    - BYE (subsequent commands ignored)

- Repeated DELETE on the same key  
  - Input:  
    - DELETE a  
    - DELETE a  
  - Output:  
    - DELETED  
    - NOT_FOUND

- High volume of operations (e.g., 10,000)  
  - Behavior: Should execute within reasonable time and without memory leaks.

- Commands with extra/missing arguments  
  - Input:  
    - PUT keyOnly  
    - GET  
    - DELETE  
  - Output: Invalid - print ERROR

- Malformed commands  
  - Input: XYZ something  
  - Output: Invalid - print ERROR

In [ ]:
# code
"""Simulate a thread-safe distributed cache using concurrent commands."""

import threading
import queue
import re


class DistributedCache:
    """Represent a thread-safe in-memory key-value cache."""

    def __init__(self):
        """Initialize the cache and the internal lock."""
        self._cache = {}
        self._lock = threading.Lock()

    def put(self, key, value):
        """Store the key-value pair in the cache."""
        with self._lock:
            self._cache[key] = value

    def get(self, key):
        """Retrieve the value for the given key from the cache."""
        with self._lock:
            return self._cache.get(key)

    def delete(self, key):
        """Remove the key from the cache if it exists."""
        with self._lock:
            if key in self._cache:
                del self._cache[key]
                return True
            return False


def validate_command(parts):
    """Validate the format and constraints of a command."""
    command = parts[0]

    if command == "PUT":
        if len(parts) != 3:
            return "ERROR"
        key, value = parts[1], parts[2]
        if not (1 <= len(key) <= 64 and key.isalnum()):
            return "ERROR"
        if not (0 <= len(value) <= 256 and re.match(r'^[\x20-\x7E]*$', value)):
            return "ERROR"
    elif command == "GET":
        if len(parts) != 2:
            return "ERROR"
        key = parts[1]
        if not (1 <= len(key) <= 64 and key.isalnum()):
            return "ERROR"
    elif command == "DELETE":
        if len(parts) != 2:
            return "ERROR"
        key = parts[1]
        if not (1 <= len(key) <= 64 and key.isalnum()):
            return "ERROR"
    elif command == "EXIT":
        if len(parts) != 1:
            return "ERROR"
    else:
        return "ERROR"

    return None


def process_command(cache, command, result_queue, index):
    """Execute a cache command and store its result in the result queue."""
    parts = command.split(' ', 2)

    validation_error = validate_command(parts)
    if validation_error:
        result_queue.put((index, validation_error))
        return

    op = parts[0]

    if op == "PUT":
        key, value = parts[1], parts[2]
        cache.put(key, value)
        result_queue.put((index, "STORED"))
    elif op == "GET":
        key = parts[1]
        value = cache.get(key)
        if value is not None:
            result_queue.put((index, value))
        else:
            result_queue.put((index, "NOT_FOUND"))
    elif op == "DELETE":
        key = parts[1]
        if cache.delete(key):
            result_queue.put((index, "DELETED"))
        else:
            result_queue.put((index, "NOT_FOUND"))
    elif op == "EXIT":
        result_queue.put((index, "BYE"))


def simulate_distributed_cache(commands: list[str]) -> list[str]:
    """
    Simulate a distributed cache with concurrent access using threads.

    Args:
        commands: A list of command strings to execute.

    Returns:
        A list of output strings in the same order as the input commands.
    """
    cache = DistributedCache()
    results = [None] * len(commands)
    result_queue = queue.Queue()
    threads = []
    exit_command_found = False

    for i, command in enumerate(commands):
        if exit_command_found:
            break

        thread = threading.Thread(
            target=process_command,
            args=(cache, command, result_queue, i)
        )
        threads.append(thread)
        thread.start()

        if command.strip().upper() == "EXIT":
            exit_command_found = True

    for thread in threads:
        thread.join()

    while not result_queue.empty():
        index, result = result_queue.get()
        results[index] = result

    final_results = []
    for result in results:
        if result is not None:
            final_results.append(result)
        else:
            # Stop collecting results after the EXIT command's output.
            if final_results and final_results[-1] == "BYE":
                break

    return final_results


In [ ]:
# tests

import unittest
from main import simulate_distributed_cache


class TestDistributedCacheSimulator(unittest.TestCase):
    """Unit tests for simulate_distributed_cache(commands: list[str]) -> list[str].
    Tests are written against the prompt and requirements, not any specific implementation.
    """

    def test_example_sequence(self):
        cmds = [
            "PUT user1 Alice",
            "PUT user2 Bob",
            "GET user1",
            "DELETE user2",
            "GET user2",
            "EXIT",
        ]
        exp = ["STORED", "STORED", "Alice", "DELETED", "NOT_FOUND", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_get_on_missing_key(self):
        cmds = ["GET missing", "EXIT"]
        exp = ["NOT_FOUND", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_delete_on_missing_key(self):
        cmds = ["DELETE nope", "EXIT"]
        exp = ["NOT_FOUND", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_put_overwrite_and_get_new_value(self):
        cmds = [
            "PUT k1 v1",
            "PUT k1 v2",  # overwrite same key
            "GET k1",
            "EXIT",
        ]
        exp = ["STORED", "STORED", "v2", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_invalid_missing_args_and_extra_args(self):
        cmds = [
            "PUT keyOnly",          # missing value
            "GET",                  # missing key
            "DELETE",               # missing key
            "PUT a b c",            # extra arg
            "GET a extra",          # extra arg
            "EXIT",
        ]
        exp = ["ERROR", "ERROR", "ERROR", "STORED", "ERROR", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_invalid_key_constraints(self):
        # Keys must be non-empty, alphanumeric, len <= 64
        long_key = "k" * 65
        cmds = [
            # missing key entirely (two spaces after PUT)
            "PUT  v",
            "PUT  ''",                  # malformed; still missing args -> ERROR
            f"PUT {long_key} v",        # too long -> ERROR
            "PUT user-1 v",             # hyphen not alphanumeric -> ERROR
            "PUT user_1 v",             # underscore not alphanumeric -> ERROR
            "PUT user1 v",              # valid
            "GET user1",
            "EXIT",
        ]
        exp = ["ERROR", "ERROR", "ERROR", "ERROR",
               "ERROR", "STORED", "v", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_invalid_value_constraints(self):
        # Values must be printable ASCII, len <= 256
        too_long_val = "x" * 257
        non_printable = "abc\x01def"
        printable = "~!@#$%^&*()_+=-[]{};:',.<>/?|\\`\""
        cmds = [
            f"PUT key1 {too_long_val}",     # too long -> ERROR
            f"PUT key2 {non_printable}",    # non-printable -> ERROR
            f"PUT key3 {printable}",        # printable OK
            "GET key3",
            "EXIT",
        ]
        exp = ["ERROR", "ERROR", "STORED", printable, "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_case_sensitivity(self):
        cmds = [
            "PUT Key1 Value",
            "GET key1",   # different case
            "GET Key1",
            "EXIT",
        ]
        exp = ["STORED", "NOT_FOUND", "Value", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_repeated_delete_behavior(self):
        cmds = [
            "PUT a A",
            "DELETE a",
            "DELETE a",
            "EXIT",
        ]
        exp = ["STORED", "DELETED", "NOT_FOUND", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_exit_mid_stream_stops_processing(self):
        cmds = [
            "PUT a A",
            "EXIT",
            "GET a",            # must be ignored after EXIT
            "DELETE a",         # ignored
        ]
        exp = ["STORED", "BYE"]  # outputs after EXIT should not appear
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_commands_with_embedded_whitespace_edges(self):
        # Ensure strict argument parsing: exactly expected count, space-separated.
        # These malformed forms must be ERROR.
        cmds = [
            "PUT   a   b",      # extra spaces but still two args: many parsers accept -> should be valid
            "GET   a",
            "PUT\tb\tc",        # tab separators should be invalid per "space-separated"
            "DELETE\tb",
            "PUT c   ",         # trailing spaces, missing value
            "GET    ",          # missing key
            "EXIT",
        ]
        exp = ["ERROR", "ERROR", "ERROR", "ERROR", "STORED", "ERROR", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_concurrent_semantics_ordered_outputs(self):
        # Even with concurrent execution per command, outputs must be returned in input order.
        cmds = [
            "PUT x 1",
            "PUT x 2",
            "GET x",
            "DELETE x",
            "GET x",
            "EXIT",
        ]
        # Deterministic expectations:
        # - Two STORED acknowledgments in order
        # - GET returns last stored value "2"
        # - DELETE removes it
        # - GET after delete returns NOT_FOUND
        exp = ["STORED", "STORED", "2", "DELETED", "NOT_FOUND", "BYE"]
        self.assertEqual(simulate_distributed_cache(cmds), exp)

    def test_high_volume_reasonable_runtime(self):
        # Create 200 operations: 100 PUTs then 100 GETs for distinct keys.
        puts = [f"PUT k{i} v{i}" for i in range(100)]
        gets = [f"GET k{i}" for i in range(100)]
        cmds = puts + gets + ["EXIT"]
        out = simulate_distributed_cache(cmds)
        # First 100 responses are STORED
        self.assertEqual(out[:100], ["STORED"] * 100)
        # Next 100 responses are v0..v99 in order
        self.assertEqual(out[100:200], [f"v{i}" for i in range(100)])
        # Last response is BYE
        self.assertEqual(out[-1], "BYE")

    def test_commands_limit_not_enforced_but_handles_many(self):
        # Not required to validate the 10,000 max, but should handle a few hundred.
        n = 300
        cmds = [f"PUT u{i} val{i}" for i in range(
            n)] + [f"GET u{i}" for i in range(n)] + ["EXIT"]
        out = simulate_distributed_cache(cmds)
        self.assertEqual(len(out), 2 * n + 1)
        self.assertTrue(all(x == "STORED" for x in out[:n]))
        self.assertEqual(out[n:2 * n], [f"val{i}" for i in range(n)])
        self.assertEqual(out[-1], "BYE")


if __name__ == "__main__":
    unittest.main()

# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/e78830da-c1c5-4963-a7bb-911c4975dbc8?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F83235%2Fview

#### Model code:

```python
# code generated by the model
import threading

class DistributedCache:
    def __init__(self):
        self.cache = {}
        self.lock = threading.Lock()
        self.exit_signal = threading.Event()

    def put(self, key, value):
        with self.lock:
            self.cache[key] = value
            return "STORED"

    def get(self, key):
        with self.lock:
            return self.cache.get(key, "NOT_FOUND")

    def delete(self, key):
        with self.lock:
            if key in self.cache:
                del self.cache[key]
                return "DELETED"
            else:
                return "NOT_FOUND"

    def exit(self):
        self.exit_signal.set()
        return "BYE"

def process_command(command, cache, result_list, result_index):
    parts = command.strip().split()
    if not parts:
        result_list[result_index] = "ERROR"
        return

    command_type = parts[0]
    if command_type == "PUT":
        if len(parts) != 3:
            result_list[result_index] = "ERROR"
        else:
            key, value = parts[1], parts[2]
            if len(key) > 64 or len(value) > 256 or not key.isalnum():
                # <Issue>: The current code does not check for an empty value string, so it would incorrectly store an empty string. The problem implies that a value must be a non-empty string.
                result_list[result_index] = "ERROR"
            else:
                result_list[result_index] = cache.put(key, value)

    elif command_type == "GET":
        if len(parts) != 2:
            result_list[result_index] = "ERROR"
        else:
            # <Issue>: No validation is performed. Example: "GET !" returns  "NOT_FOUND" instead of "ERROR"
            key = parts[1]
            result_list[result_index] = cache.get(key)

    elif command_type == "DELETE":
        if len(parts) != 2:
            result_list[result_index] = "ERROR"
        else:
            # <Issue>: No validation is performed. Example: "DELETE !" returns
            # "NOT_FOUND" instead of "ERROR"
            key = parts[1]
            result_list[result_index] = cache.delete(key)

    elif command_type == "EXIT":
        if len(parts) != 1:
            result_list[result_index] = "ERROR"
        else:
            result_list[result_index] = cache.exit()

    else:
        result_list[result_index] = "ERROR"

def simulate_distributed_cache(commands):
    cache = DistributedCache()
    result_list = [None] * len(commands)
    threads = []

    for index, command in enumerate(commands):
        if cache.exit_signal.is_set():
            break
        thread = threading.Thread(target=process_command, args=(command, cache, result_list, index))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    return result_list

# Example Usage:
# commands = [
#     "PUT user1 Alice",
#     "PUT user2 Bob",
#     "GET user1",
#     "DELETE user2",
#     "GET user2",
#     "EXIT"
# ]
# print(simulate_distributed_cache(commands))
```